#### Clean data about crashes where pedestrians or cyclists were injured or killed in Brooklyn
##### Remove entries without coordinates

In [1]:
import warnings, os
import pandas as pd
import numpy as np
import geopandas as gpd
%matplotlib inline

In [2]:
# Load crash data file
crashes = gpd.read_file("../Raw Data/PedestriansCyclistsKilledInjured.csv")

In [3]:
df = pd.read_csv('../Raw Data/PedestriansCyclistsKilledInjured.csv')

In [4]:
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,"(40.623104, -73.95809)",EAST 18 STREET,AVENUE K,NaN,...,NaN,NaN,NaN,NaN,4486516,Sedan,NaN,NaN,NaN,NaN
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,"(40.687534, -73.9775)",FULTON STREET,SAINT FELIX STREET,NaN,...,Unspecified,NaN,NaN,NaN,4487052,Sedan,Bike,NaN,NaN,NaN
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,"(40.65011, -73.930214)",UTICA AVENUE,SNYDER AVENUE,NaN,...,NaN,NaN,NaN,NaN,4522136,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,"(40.59207, -73.96299)",EAST 7 STREET,CRAWFORD AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,"(40.640835, -73.98967)",12 AVENUE,41 STREET,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4485355,Sedan,Bike,NaN,NaN,NaN


In [5]:
# Keep only the specified columns
columns_to_keep = [
    'CRASH DATE',
    'CRASH TIME', 
    'BOROUGH',
    'ZIP CODE',
    'LATITUDE',
    'LONGITUDE',
    'NUMBER OF PERSONS INJURED',
    'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED',
    'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED',
    'NUMBER OF CYCLIST KILLED'
]

df_filtered = df[columns_to_keep]

In [6]:
df_filtered.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,1,0.0,1,0,0,0
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,1,0.0,0,0,1,0
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,1,0.0,1,0,0,0
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,1,0.0,0,0,1,0
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,1,0.0,0,0,1,0


In [7]:
# Remove rows with 0,0 coordinates or null values
df_filtered = df_filtered[
    (df_filtered['LATITUDE'] != 0) & 
    (df_filtered['LONGITUDE'] != 0) &
    (df_filtered['LATITUDE'].notna()) &
    (df_filtered['LONGITUDE'].notna())
]

print(f"Removed {len(df) - len(df_filtered)} rows with invalid coordinates")
print(f"Remaining rows: {len(df_filtered)}")

Removed 1131 rows with invalid coordinates
Remaining rows: 50493


#### Create GeoDataFrame

In [8]:
# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_filtered, 
    geometry=gpd.points_from_xy(df_filtered['LONGITUDE'], df_filtered['LATITUDE']),
    crs='EPSG:4326'
)

In [9]:
gdf = gdf.to_crs('EPSG:2263')

In [10]:
gdf.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,geometry
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,1,0.0,1,0,0,0,POINT (995884.243 166292.752)
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,1,0.0,0,0,1,0,POINT (990489.985 189764.409)
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,1,0.0,1,0,0,0,POINT (1003614.798 176136.698)
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,1,0.0,0,0,1,0,POINT (994528.777 154985.651)
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,1,0.0,0,0,1,0,POINT (987116.853 172750.017)


#### Create a column called CRASH_DATETIME in the datetime format

In [11]:
# Check data type of CRASH DATE column
print(df_filtered['CRASH DATE'].dtype)

object


In [12]:
# Combine the original string columns before converting
gdf['CRASH_DATETIME'] = pd.to_datetime(gdf['CRASH DATE'].astype(str) + ' ' + gdf['CRASH TIME'].astype(str))

In [13]:
print(gdf['CRASH_DATETIME'].head())
print(gdf['CRASH_DATETIME'].dtype)



0   2021-12-14 17:31:00
1   2021-12-14 12:54:00
2   2022-04-12 19:56:00
3   2021-12-09 20:20:00
4   2021-12-09 23:15:00
Name: CRASH_DATETIME, dtype: datetime64[ns]
datetime64[ns]


In [14]:
# See date range
print("Date range:")
print("Earliest:", gdf['CRASH_DATETIME'].min())
print("Latest:", gdf['CRASH_DATETIME'].max())

Date range:
Earliest: 2012-07-01 01:10:00
Latest: 2025-06-29 19:45:00


In [15]:
gdf.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,geometry,CRASH_DATETIME
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,1,0.0,1,0,0,0,POINT (995884.243 166292.752),2021-12-14 17:31:00
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,1,0.0,0,0,1,0,POINT (990489.985 189764.409),2021-12-14 12:54:00
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,1,0.0,1,0,0,0,POINT (1003614.798 176136.698),2022-04-12 19:56:00
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,1,0.0,0,0,1,0,POINT (994528.777 154985.651),2021-12-09 20:20:00
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,1,0.0,0,0,1,0,POINT (987116.853 172750.017),2021-12-09 23:15:00


#### Creating a weekly rate for 11 meter square clusters.
##### Since most of the crashes happen at intersections anyway, this will cluster crashes at an 'intersection'

In [16]:
# Check for null values in all columns
print("Null values per column:")
print(gdf.isnull().sum())

# Show percentage of nulls
print("\nPercentage of nulls per column:")
print((gdf.isnull().sum() / len(gdf) * 100).round(2))

# Get a summary of data types and non-null counts
print("\nColumn info:")
print(gdf.info())

# Check for empty strings (sometimes appears as valid data but is actually missing)
print("\nEmpty string values per column:")
for col in gdf.columns:
    if gdf[col].dtype == 'object':  # Only check text columns
        empty_count = (gdf[col] == '').sum()
        if empty_count > 0:
            print(f"{col}: {empty_count} empty strings")

# Check for any rows that have nulls in critical columns
critical_columns = ['CRASH_DATETIME', 'LATITUDE', 'LONGITUDE', 
                   'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
                   'NUMBER OF CYCLIST KILLED', 'NUMBER OF CYCLIST INJURED']

print(f"\nRows with nulls in critical columns:")
for col in critical_columns:
    if col in gdf.columns:
        null_count = gdf[col].isnull().sum()
        print(f"{col}: {null_count} nulls")

Null values per column:
CRASH DATE                       0
CRASH TIME                       0
BOROUGH                          0
ZIP CODE                         0
LATITUDE                         0
LONGITUDE                        0
NUMBER OF PERSONS INJURED        0
NUMBER OF PERSONS KILLED         1
NUMBER OF PEDESTRIANS INJURED    0
NUMBER OF PEDESTRIANS KILLED     0
NUMBER OF CYCLIST INJURED        0
NUMBER OF CYCLIST KILLED         0
geometry                         0
CRASH_DATETIME                   0
dtype: int64

Percentage of nulls per column:
CRASH DATE                       0.0
CRASH TIME                       0.0
BOROUGH                          0.0
ZIP CODE                         0.0
LATITUDE                         0.0
LONGITUDE                        0.0
NUMBER OF PERSONS INJURED        0.0
NUMBER OF PERSONS KILLED         0.0
NUMBER OF PEDESTRIANS INJURED    0.0
NUMBER OF PEDESTRIANS KILLED     0.0
NUMBER OF CYCLIST INJURED        0.0
NUMBER OF CYCLIST KILLED         

In [17]:
# Create intersection ID (round to ~11 meter precision)
gdf['intersection_id'] = (gdf['LATITUDE'].round(4).astype(str) + '_' + 
                         gdf['LONGITUDE'].round(4).astype(str))

In [18]:
# Create year-week field
gdf['year_week'] = gdf['CRASH_DATETIME'].dt.strftime('%Y-W%U')


In [21]:
# Create total casualties column
gdf['total_casualties'] = (gdf['NUMBER OF PEDESTRIANS KILLED'] + 
                          gdf['NUMBER OF PEDESTRIANS INJURED'] + 
                          gdf['NUMBER OF CYCLIST KILLED'] + 
                          gdf['NUMBER OF CYCLIST INJURED'])

In [22]:
# Calculate weekly rate per intersection
weekly_intersection_stats = gdf.groupby(['year_week', 'intersection_id'])['total_casualties'].sum().reset_index()
weekly_intersection_stats.rename(columns={'total_casualties': 'weekly_intersection_rate'}, inplace=True)


In [23]:
# Merge back to original data
gdf = gdf.merge(weekly_intersection_stats, on=['year_week', 'intersection_id'], how='left')

In [24]:
# Check the results
print("New columns created:")
print(gdf[['year_week', 'intersection_id', 'total_casualties', 'weekly_intersection_rate']].head())

New columns created:
  year_week   intersection_id  total_casualties  weekly_intersection_rate
0  2021-W50  40.6231_-73.9581                 1                         1
1  2021-W50  40.6875_-73.9775                 1                         1
2  2022-W15  40.6501_-73.9302                 1                         1
3  2021-W49   40.5921_-73.963                 1                         1
4  2021-W49  40.6408_-73.9897                 1                         1


In [25]:
# Basic min/max
print("Weekly intersection rate range:")
print(f"Minimum: {gdf['weekly_intersection_rate'].min()}")
print(f"Maximum: {gdf['weekly_intersection_rate'].max()}")

# More detailed statistics
print("\nDetailed statistics:")
print(gdf['weekly_intersection_rate'].describe())

# See the actual records with min and max values
print("\nRow with minimum rate:")
print(gdf[gdf['weekly_intersection_rate'] == gdf['weekly_intersection_rate'].min()][['intersection_id', 'year_week', 'weekly_intersection_rate']].head(1))

print("\nRow with maximum rate:")
print(gdf[gdf['weekly_intersection_rate'] == gdf['weekly_intersection_rate'].max()][['intersection_id', 'year_week', 'weekly_intersection_rate']].head(1))

# Check value counts to see distribution
print("\nValue counts (top 10):")
print(gdf['weekly_intersection_rate'].value_counts().head(10))

Weekly intersection rate range:
Minimum: 1
Maximum: 9

Detailed statistics:
count    50493.000000
mean         1.035114
std          0.220491
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          9.000000
Name: weekly_intersection_rate, dtype: float64

Row with minimum rate:
    intersection_id year_week  weekly_intersection_rate
0  40.6231_-73.9581  2021-W50                         1

Row with maximum rate:
        intersection_id year_week  weekly_intersection_rate
42811  40.6517_-73.9304  2013-W12                         9

Value counts (top 10):
weekly_intersection_rate
1    48979
2     1313
3      166
4       28
8        3
6        1
5        1
7        1
9        1
Name: count, dtype: int64


#### Cyclists intersection rate

In [26]:
# Create cyclist casualties column (injuries + deaths)
gdf['cyclist_casualties'] = (gdf['NUMBER OF CYCLIST KILLED'] + 
                            gdf['NUMBER OF CYCLIST INJURED'])

# Calculate weekly cyclist rate per intersection
weekly_cyclist_stats = gdf.groupby(['year_week', 'intersection_id'])['cyclist_casualties'].sum().reset_index()
weekly_cyclist_stats.rename(columns={'cyclist_casualties': 'weekly_cyclist_rate'}, inplace=True)

# Merge back to original data
gdf = gdf.merge(weekly_cyclist_stats, on=['year_week', 'intersection_id'], how='left')

# Check the distribution
print("Weekly cyclist rate distribution:")
print(gdf['weekly_cyclist_rate'].value_counts().sort_index())

# Check min/max
print(f"\nCyclist weekly rate range:")
print(f"Minimum: {gdf['weekly_cyclist_rate'].min()}")
print(f"Maximum: {gdf['weekly_cyclist_rate'].max()}")

# Show some examples
print(f"\nSample of highest cyclist rates:")
print(gdf[gdf['weekly_cyclist_rate'] > 1][['intersection_id', 'year_week', 'weekly_cyclist_rate']].drop_duplicates().head())

Weekly cyclist rate distribution:
weekly_cyclist_rate
0    32019
1    18108
2      353
3       11
4        2
Name: count, dtype: int64

Cyclist weekly rate range:
Minimum: 0
Maximum: 4

Sample of highest cyclist rates:
       intersection_id year_week  weekly_cyclist_rate
503   40.6515_-74.0073  2021-W19                    2
560   40.6547_-73.9602  2021-W38                    2
561   40.6931_-73.9951  2021-W38                    2
803   40.6624_-73.9652  2021-W21                    2
1016  40.6468_-74.0086  2021-W23                    2


In [27]:
# Find intersections with most total cyclist casualties
worst_total = gdf.groupby('intersection_id').agg({
    'cyclist_casualties': 'sum',
    'weekly_cyclist_rate': 'max',
    'LATITUDE': 'first',
    'LONGITUDE': 'first'
}).reset_index().sort_values('cyclist_casualties', ascending=False)

print("Worst intersections by total cyclist casualties:")
print(worst_total.head(10))

Worst intersections by total cyclist casualties:
        intersection_id  cyclist_casualties  weekly_cyclist_rate   LATITUDE  \
11557  40.6818_-73.9675                  33                    3  40.681767   
13371  40.6961_-73.9871                  32                    2  40.696130   
14746  40.7112_-73.9489                  30                    1  40.711166   
13551  40.6977_-73.9677                  28                    2  40.697716   
14813   40.711_-73.9511                  27                    2  40.710957   
10980   40.6787_-73.953                  24                    1  40.678722   
14871   40.712_-73.9407                  23                    1  40.711960   
14775  40.7116_-73.9439                  23                    1  40.711643   
7466   40.6549_-73.9619                  22                    2  40.654880   
14897  40.7134_-73.9348                  21                    1  40.713370   

       LONGITUDE  
11557 -73.967540  
13371 -73.987114  
14746 -73.948900  
13551

#### Export file

In [ ]:
os.makedirs('Clean Data', exist_ok=True)
gdf.to_file('brooklyn_cyclist_crashes_2012_2024_WIR.geojson', driver='GeoJSON')

#### Top 10 worst spots

In [ ]:
# Create summary of intersections with cyclist casualties
intersection_summary = gdf.groupby('intersection_id').agg({
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum', 
    'cyclist_casualties': 'sum',
    'LATITUDE': 'first',
    'LONGITUDE': 'first'
}).reset_index()

# Rename columns for clarity
intersection_summary.rename(columns={
    'NUMBER OF CYCLIST KILLED': 'cyclist_deaths',
    'NUMBER OF CYCLIST INJURED': 'cyclist_injuries',
    'cyclist_casualties': 'total_casualties'
}, inplace=True)

# Get top 10 by total casualties
top_10 = intersection_summary.sort_values('total_casualties', ascending=False).head(10)

# Create geometry for GeoDataFrame
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(top_10['LONGITUDE'], top_10['LATITUDE'])]

# Create GeoDataFrame
top_10_gdf = gpd.GeoDataFrame(top_10, geometry=geometry, crs='EPSG:2263')

# Select only the columns you want
columns_to_keep = ['intersection_id', 'cyclist_injuries', 'cyclist_deaths', 
                   'total_casualties', 'LATITUDE', 'LONGITUDE', 'geometry']
top_10_final = top_10_gdf[columns_to_keep]

# Export to GeoJSON
# top_10_final.to_file('top_10_cyclist_intersections.geojson', driver='GeoJSON')

# Display the results
print("Top 10 Intersections by Cyclist Casualties:")
print(top_10_final.drop('geometry', axis=1))

print(f"\nExported {len(top_10_final)} intersections to 'top_10_cyclist_intersections.geojson'")

Top 10 Intersections by Cyclist Casualties:
        intersection_id  cyclist_injuries  cyclist_deaths  total_casualties  \
11557  40.6818_-73.9675                33               0                33   
13371  40.6961_-73.9871                32               0                32   
14746  40.7112_-73.9489                30               0                30   
13551  40.6977_-73.9677                28               0                28   
14813   40.711_-73.9511                27               0                27   
10980   40.6787_-73.953                24               0                24   
14871   40.712_-73.9407                23               0                23   
14775  40.7116_-73.9439                23               0                23   
7466   40.6549_-73.9619                22               0                22   
14897  40.7134_-73.9348                21               0                21   

        LATITUDE  LONGITUDE  
11557  40.681767 -73.967540  
13371  40.696130 -73.98711

#### Top 20 worst spots

In [ ]:
# Create summary of intersections with cyclist casualties
intersection_summary = gdf.groupby('intersection_id').agg({
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum', 
    'cyclist_casualties': 'sum',
    'LATITUDE': 'first',
    'LONGITUDE': 'first'
}).reset_index()

# Rename columns for clarity
intersection_summary.rename(columns={
    'NUMBER OF CYCLIST KILLED': 'cyclist_deaths',
    'NUMBER OF CYCLIST INJURED': 'cyclist_injuries',
    'cyclist_casualties': 'total_casualties'
}, inplace=True)

# Get top 20 by total casualties
top_20 = intersection_summary.sort_values('total_casualties', ascending=False).head(20)

# Create geometry for GeoDataFrame
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(top_20['LONGITUDE'], top_20['LATITUDE'])]

# Create GeoDataFrame
top_20_gdf = gpd.GeoDataFrame(top_20, geometry=geometry, crs='EPSG:2263')

# Select only the columns you want
columns_to_keep = ['intersection_id', 'cyclist_injuries', 'cyclist_deaths', 
                   'total_casualties', 'LATITUDE', 'LONGITUDE', 'geometry']
top_20_final = top_20_gdf[columns_to_keep]

# Export to GeoJSON
# top_20_final.to_file('top_20_cyclist_intersections.geojson', driver='GeoJSON')

# Display the results
print("Top 20 Intersections by Cyclist Casualties:")
print(top_20_final.drop('geometry', axis=1))

print(f"\nExported {len(top_20_final)} intersections to 'top_10_cyclist_intersections.geojson'")

Top 20 Intersections by Cyclist Casualties:
        intersection_id  cyclist_injuries  cyclist_deaths  total_casualties  \
11557  40.6818_-73.9675                33               0                33   
13371  40.6961_-73.9871                32               0                32   
14746  40.7112_-73.9489                30               0                30   
13551  40.6977_-73.9677                28               0                28   
14813   40.711_-73.9511                27               0                27   
10980   40.6787_-73.953                24               0                24   
14871   40.712_-73.9407                23               0                23   
14775  40.7116_-73.9439                23               0                23   
7466   40.6549_-73.9619                22               0                22   
14897  40.7134_-73.9348                21               0                21   
14313  40.7055_-73.9502                21               0                21   
14715   

#### Top 20 worst weeks 

In [44]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load your data - replace with your actual GeoDataFrame variable name
# If you already have the data loaded as 'gdf', skip this line:
# gdf = gpd.read_file('brooklyn_crashes_with_cyclist_rates.geojson')  # Use your actual filename

# Or if you already have gdf loaded in memory, just continue with:

# Filter to Brooklyn bounds (to avoid Pennsylvania coordinates)
brooklyn_bounds = (
    (gdf['LATITUDE'] >= 40.57) & 
    (gdf['LATITUDE'] <= 40.74) & 
    (gdf['LONGITUDE'] >= -74.04) & 
    (gdf['LONGITUDE'] <= -73.86)
)
gdf_brooklyn = gdf[brooklyn_bounds]

# Create cyclist casualties column if not exists
if 'cyclist_casualties' not in gdf_brooklyn.columns:
    gdf_brooklyn['cyclist_casualties'] = (gdf_brooklyn['NUMBER OF CYCLIST KILLED'] + 
                                         gdf_brooklyn['NUMBER OF CYCLIST INJURED'])

# Group by intersection and calculate totals
top_20_intersections = gdf_brooklyn.groupby('intersection_id').agg({
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum', 
    'cyclist_casualties': 'sum',
    'LATITUDE': 'first',
    'LONGITUDE': 'first'
}).reset_index()

# Rename columns for clarity
top_20_intersections.rename(columns={
    'NUMBER OF CYCLIST KILLED': 'cyclist_deaths',
    'NUMBER OF CYCLIST INJURED': 'cyclist_injuries',
    'cyclist_casualties': 'total_casualties'
}, inplace=True)

# Get top 20 by total casualties
top_20 = top_20_intersections.sort_values('total_casualties', ascending=False).head(20)

# Create geometry for GeoDataFrame
geometry = [Point(xy) for xy in zip(top_20['LONGITUDE'], top_20['LATITUDE'])]
top_20_gdf = gpd.GeoDataFrame(top_20, geometry=geometry, crs='EPSG:4326')

# Display results
print("Top 20 Worst Intersections for Cyclists:")
print(top_20)

# Export to GeoJSON
top_20_gdf.to_file('top_20_worst_cyclist_intersections.geojson', driver='GeoJSON')
print(f"\nExported {len(top_20_gdf)} intersections to 'top_20_worst_cyclist_intersections.geojson'")

# Summary stats
print(f"\nSummary:")
print(f"Total cyclist deaths at top 20 intersections: {top_20['cyclist_deaths'].sum()}")
print(f"Total cyclist injuries at top 20 intersections: {top_20['cyclist_injuries'].sum()}")
print(f"Total casualties at top 20 intersections: {top_20['total_casualties'].sum()}")

Top 20 Worst Intersections for Cyclists:
        intersection_id  cyclist_deaths  cyclist_injuries  total_casualties  \
11538  40.6818_-73.9675               0                33                33   
13352  40.6961_-73.9871               0                32                32   
14726  40.7112_-73.9489               0                30                30   
13532  40.6977_-73.9677               0                28                28   
14793   40.711_-73.9511               0                27                27   
10962   40.6787_-73.953               0                24                24   
14755  40.7116_-73.9439               0                23                23   
14851   40.712_-73.9407               0                23                23   
7459   40.6549_-73.9619               0                22                22   
14695    40.7108_-73.96               0                21                21   
14877  40.7134_-73.9348               0                21                21   
14293  40.7

In [33]:
# Get details for the worst week
worst_week = top_20_weeks.iloc[0]['year_week']
worst_week_details = gdf[gdf['year_week'] == worst_week]

print(f"\nDetails for worst week ({worst_week}):")
worst_week_crashes = worst_week_details[worst_week_details['cyclist_casualties'] > 0]
print(f"Number of cyclist crashes: {len(worst_week_crashes)}")
print(f"Intersections involved:")
print(worst_week_crashes[['intersection_id', 'cyclist_casualties', 'LATITUDE', 'LONGITUDE']].head(10))

# Create export data with more details
worst_week_export = worst_week_crashes[['intersection_id', 'cyclist_casualties', 'LATITUDE', 'LONGITUDE', 
                                       'NUMBER OF CYCLIST KILLED', 'NUMBER OF CYCLIST INJURED',
                                       'CRASH_DATETIME', 'BOROUGH', 'ZIP CODE']].copy()

# Rename columns for clarity
worst_week_export.rename(columns={
    'NUMBER OF CYCLIST KILLED': 'cyclist_deaths',
    'NUMBER OF CYCLIST INJURED': 'cyclist_injuries',
    'ZIP CODE': 'zip_code'
}, inplace=True)

# Export as CSV
csv_filename = f'worst_week_{worst_week}_cyclist_crashes.csv'
worst_week_export.to_csv(csv_filename, index=False)
print(f"\nExported {len(worst_week_export)} crashes to '{csv_filename}'")

# Export as GeoJSON
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(worst_week_export['LONGITUDE'], worst_week_export['LATITUDE'])]
worst_week_gdf = gpd.GeoDataFrame(worst_week_export, geometry=geometry, crs='EPSG:4326')

geojson_filename = f'worst_week_{worst_week}_cyclist_crashes.geojson'
worst_week_gdf.to_file(geojson_filename, driver='GeoJSON')
print(f"Exported {len(worst_week_gdf)} crashes to '{geojson_filename}'")

# Summary stats for the worst week
print(f"\nSummary for week {worst_week}:")
print(f"Total cyclist casualties: {worst_week_crashes['cyclist_casualties'].sum()}")
print(f"Total deaths: {worst_week_crashes['NUMBER OF CYCLIST KILLED'].sum()}")
print(f"Total injuries: {worst_week_crashes['NUMBER OF CYCLIST INJURED'].sum()}")
print(f"Unique intersections affected: {worst_week_crashes['intersection_id'].nunique()}")


Details for worst week (2018-W33):
Number of cyclist crashes: 60
Intersections involved:
        intersection_id  cyclist_casualties   LATITUDE  LONGITUDE
20311  40.6539_-74.0082                   1  40.653873 -74.008156
20315   40.651_-73.9457                   1  40.651005 -73.945710
20317   40.709_-73.9223                   1  40.709003 -73.922340
20330  40.6061_-73.9656                   1  40.606120 -73.965650
20331   40.7241_-73.951                   1  40.724100 -73.950990
20334  40.6988_-73.9533                   1  40.698837 -73.953270
20336  40.6061_-74.0074                   1  40.606102 -74.007440
20344  40.6805_-73.9522                   1  40.680534 -73.952156
20348  40.5993_-73.9523                   1  40.599293 -73.952286
20352  40.6503_-73.9662                   1  40.650295 -73.966156

Exported 60 crashes to 'worst_week_2018-W33_cyclist_crashes.csv'
Exported 60 crashes to 'worst_week_2018-W33_cyclist_crashes.geojson'

Summary for week 2018-W33:
Total cyclist casualt

#### casualties per year

In [35]:
# Extract year from year_week and sum cyclist casualties
casualties_per_year = gdf.groupby(gdf['year_week'].str[:4]).agg({
    'cyclist_casualties': 'sum',
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum'
}).reset_index()

# Rename columns for clarity
casualties_per_year.rename(columns={
    'year_week': 'year',
    'cyclist_casualties': 'total_casualties',
    'NUMBER OF CYCLIST KILLED': 'cyclist_deaths',
    'NUMBER OF CYCLIST INJURED': 'cyclist_injuries'
}, inplace=True)

# Convert year to integer
casualties_per_year['year'] = casualties_per_year['year'].astype(int)

# Sort by year
casualties_per_year = casualties_per_year.sort_values('year')

print("Cyclist Casualties per Year:")
print(casualties_per_year)

# Calculate trends
print(f"\nWorst year: {casualties_per_year.loc[casualties_per_year['total_casualties'].idxmax(), 'year']} with {casualties_per_year['total_casualties'].max()} casualties")
print(f"Best year: {casualties_per_year.loc[casualties_per_year['total_casualties'].idxmin(), 'year']} with {casualties_per_year['total_casualties'].min()} casualties")
print(f"Average per year: {casualties_per_year['total_casualties'].mean():.1f} casualties")

# Export to CSV
casualties_per_year.to_csv('cyclist_casualties_per_year.csv', index=False)
print("Exported to 'cyclist_casualties_per_year.csv'")

Cyclist Casualties per Year:
    year  total_casualties  cyclist_deaths  cyclist_injuries
0   2012               881               2               879
1   2013              1543               4              1539
2   2014              1414               4              1410
3   2015              1518               4              1514
4   2016              1306               4              1302
5   2017              1288               7              1281
6   2018              1350               2              1348
7   2019              1390              13              1377
8   2020              1529               8              1521
9   2021              1333               2              1331
10  2022              1361               4              1357
11  2023              1481               9              1472
12  2024              1469               8              1461
13  2025               653               0               653

Worst year: 2013 with 1543 casualties
Best year: 2025 w

In [36]:
# More robust method using actual dates
gdf['year'] = gdf['CRASH_DATETIME'].dt.year
yearly_summary = gdf.groupby('year').agg({
    'cyclist_casualties': 'sum',
    'NUMBER OF CYCLIST KILLED': 'sum', 
    'NUMBER OF CYCLIST INJURED': 'sum'
}).reset_index()

yearly_summary.rename(columns={
    'cyclist_casualties': 'total_casualties',
    'NUMBER OF CYCLIST KILLED': 'cyclist_deaths',
    'NUMBER OF CYCLIST INJURED': 'cyclist_injuries'
}, inplace=True)

print("Yearly Cyclist Casualties (using datetime):")
print(yearly_summary)

Yearly Cyclist Casualties (using datetime):
    year  total_casualties  cyclist_deaths  cyclist_injuries
0   2012               881               2               879
1   2013              1543               4              1539
2   2014              1414               4              1410
3   2015              1518               4              1514
4   2016              1306               4              1302
5   2017              1288               7              1281
6   2018              1350               2              1348
7   2019              1390              13              1377
8   2020              1529               8              1521
9   2021              1333               2              1331
10  2022              1361               4              1357
11  2023              1481               9              1472
12  2024              1469               8              1461
13  2025               653               0               653


In [37]:
#### Cyclist deaths 2025

In [40]:
# Quick summary
print("Cyclist deaths by year (recent years):")
recent_deaths = gdf[gdf['CRASH_DATETIME'].dt.year >= 2022].groupby(gdf['CRASH_DATETIME'].dt.year)['NUMBER OF CYCLIST KILLED'].sum()
print(recent_deaths)

Cyclist deaths by year (recent years):
CRASH_DATETIME
2022    4
2023    9
2024    8
2025    0
Name: NUMBER OF CYCLIST KILLED, dtype: int64


#### Top 5 worst intersections in the last 5 years

In [28]:
# Filter to last 5 years (2020-2025)
last_5_years = gdf[gdf['CRASH_DATETIME'].dt.year >= 2020]

# Filter to Brooklyn bounds (you already have this data, but just to be safe)
brooklyn_bounds = (
    (last_5_years['LATITUDE'] >= 40.57) & 
    (last_5_years['LATITUDE'] <= 40.74) & 
    (last_5_years['LONGITUDE'] >= -74.04) & 
    (last_5_years['LONGITUDE'] <= -73.86)
)
gdf_recent = last_5_years[brooklyn_bounds]

print(f"Filtered to {len(gdf_recent)} crashes from 2020-2025 in Brooklyn")

# Group by intersection and calculate totals for recent years
top_5_recent = gdf_recent.groupby('intersection_id').agg({
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum', 
    'cyclist_casualties': 'sum',
    'LATITUDE': 'first',
    'LONGITUDE': 'first',
    'CRASH_DATETIME': ['min', 'max', 'count']  # First crash, last crash, total crashes
}).reset_index()

# Flatten column names
top_5_recent.columns = ['intersection_id', 'cyclist_deaths', 'cyclist_injuries', 
                       'total_casualties', 'LATITUDE', 'LONGITUDE', 
                       'first_crash', 'last_crash', 'total_crashes']

# Get top 5 by total casualties in recent years
top_5 = top_5_recent.sort_values('total_casualties', ascending=False).head(5)

# Create geometry for GeoDataFrame
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(top_5['LONGITUDE'], top_5['LATITUDE'])]
top_5_gdf = gpd.GeoDataFrame(top_5, geometry=geometry, crs='EPSG:4326')

# Display detailed results
print("\nTop 5 Worst Intersections for Cyclists (2020-2025):")
print("=" * 60)
for i, (idx, row) in enumerate(top_5.iterrows(), 1):
    print(f"{i}. Intersection: {row['intersection_id']}")
    print(f"   Location: {row['LATITUDE']:.4f}, {row['LONGITUDE']:.4f}")
    print(f"   Total casualties: {row['total_casualties']}")
    print(f"   Deaths: {row['cyclist_deaths']}")
    print(f"   Injuries: {row['cyclist_injuries']}")
    print(f"   Total crashes: {row['total_crashes']}")
    print(f"   Period: {row['first_crash'].strftime('%Y-%m-%d')} to {row['last_crash'].strftime('%Y-%m-%d')}")
    print()

# Export to GeoJSON
top_5_gdf.to_file('top_5_worst_cyclist_intersections_2020_2025.geojson', driver='GeoJSON')
print(f"Exported to 'top_5_worst_cyclist_intersections_2020_2025.geojson'")

# Summary stats
print(f"Summary for Top 5 (2020-2025):")
print(f"Total cyclist deaths: {top_5['cyclist_deaths'].sum()}")
print(f"Total cyclist injuries: {top_5['cyclist_injuries'].sum()}")
print(f"Total casualties: {top_5['total_casualties'].sum()}")
print(f"Total crashes: {top_5['total_crashes'].sum()}")

Filtered to 18926 crashes from 2020-2025 in Brooklyn

Top 5 Worst Intersections for Cyclists (2020-2025):
1. Intersection: 40.7108_-73.96
   Location: 40.7108, -73.9600
   Total casualties: 13
   Deaths: 0
   Injuries: 13
   Total crashes: 14
   Period: 2020-05-03 to 2024-10-16

2. Intersection: 40.6515_-74.0073
   Location: 40.6515, -74.0073
   Total casualties: 13
   Deaths: 0
   Injuries: 13
   Total crashes: 11
   Period: 2021-05-10 to 2024-10-23

3. Intersection: 40.6977_-73.9677
   Location: 40.6977, -73.9677
   Total casualties: 12
   Deaths: 0
   Injuries: 12
   Total crashes: 11
   Period: 2020-09-28 to 2024-12-12

4. Intersection: 40.6661_-73.9922
   Location: 40.6661, -73.9922
   Total casualties: 11
   Deaths: 0
   Injuries: 11
   Total crashes: 11
   Period: 2020-09-18 to 2024-09-07

5. Intersection: 40.6728_-73.9867
   Location: 40.6728, -73.9867
   Total casualties: 11
   Deaths: 0
   Injuries: 11
   Total crashes: 11
   Period: 2022-04-19 to 2025-05-18

Exported to 'top

#### Top 5 worst intersections in the last 5 years using 20m intersections

In [29]:
# Filter to last 5 years (2020-2025)
last_5_years = gdf[gdf['CRASH_DATETIME'].dt.year >= 2020]

# Filter to Brooklyn bounds
brooklyn_bounds = (
    (last_5_years['LATITUDE'] >= 40.57) & 
    (last_5_years['LATITUDE'] <= 40.74) & 
    (last_5_years['LONGITUDE'] >= -74.04) & 
    (last_5_years['LONGITUDE'] <= -73.86)
)
gdf_recent = last_5_years[brooklyn_bounds]

print(f"Filtered to {len(gdf_recent)} crashes from 2020-2025 in Brooklyn")

# Create larger intersection ID (round to ~20 meter precision)
gdf_recent['intersection_id_20m'] = (gdf_recent['LATITUDE'].round(3).astype(str) + '_' + 
                                    gdf_recent['LONGITUDE'].round(3).astype(str))

# Group by larger intersection and calculate totals for recent years
top_5_recent_20m = gdf_recent.groupby('intersection_id_20m').agg({
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum', 
    'cyclist_casualties': 'sum',
    'LATITUDE': 'first',
    'LONGITUDE': 'first',
    'CRASH_DATETIME': ['min', 'max', 'count']  # First crash, last crash, total crashes
}).reset_index()

# Flatten column names
top_5_recent_20m.columns = ['intersection_id_20m', 'cyclist_deaths', 'cyclist_injuries', 
                           'total_casualties', 'LATITUDE', 'LONGITUDE', 
                           'first_crash', 'last_crash', 'total_crashes']

# Get top 5 by total casualties in recent years
top_5_20m = top_5_recent_20m.sort_values('total_casualties', ascending=False).head(5)

# Create geometry for GeoDataFrame
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(top_5_20m['LONGITUDE'], top_5_20m['LATITUDE'])]
top_5_gdf_20m = gpd.GeoDataFrame(top_5_20m, geometry=geometry, crs='EPSG:4326')

# Display detailed results
print("\nTop 5 Worst Intersections for Cyclists (2020-2025) - 20m clusters:")
print("=" * 70)
for i, (idx, row) in enumerate(top_5_20m.iterrows(), 1):
    print(f"{i}. Intersection: {row['intersection_id_20m']}")
    print(f"   Location: {row['LATITUDE']:.3f}, {row['LONGITUDE']:.3f}")
    print(f"   Total casualties: {row['total_casualties']}")
    print(f"   Deaths: {row['cyclist_deaths']}")
    print(f"   Injuries: {row['cyclist_injuries']}")
    print(f"   Total crashes: {row['total_crashes']}")
    print(f"   Period: {row['first_crash'].strftime('%Y-%m-%d')} to {row['last_crash'].strftime('%Y-%m-%d')}")
    print()

# Export to GeoJSON
top_5_gdf_20m.to_file('top_5_worst_cyclist_intersections_2020_2025_20m.geojson', driver='GeoJSON')
print(f"Exported to 'top_5_worst_cyclist_intersections_2020_2025_20m.geojson'")

# Summary stats
print(f"Summary for Top 5 (2020-2025, 20m clusters):")
print(f"Total cyclist deaths: {top_5_20m['cyclist_deaths'].sum()}")
print(f"Total cyclist injuries: {top_5_20m['cyclist_injuries'].sum()}")
print(f"Total casualties: {top_5_20m['total_casualties'].sum()}")
print(f"Total crashes: {top_5_20m['total_crashes'].sum()}")

# Compare with 11m precision
print(f"\nComparison: 20m vs 11m precision")
print(f"20m clusters group more crashes together, showing broader danger zones")
print(f"11m clusters are more precise intersection-specific")

Filtered to 18926 crashes from 2020-2025 in Brooklyn

Top 5 Worst Intersections for Cyclists (2020-2025) - 20m clusters:
1. Intersection: 40.711_-73.96
   Location: 40.711, -73.960
   Total casualties: 18
   Deaths: 0
   Injuries: 18
   Total crashes: 22
   Period: 2020-02-05 to 2025-06-28

2. Intersection: 40.652_-74.007
   Location: 40.652, -74.007
   Total casualties: 15
   Deaths: 0
   Injuries: 15
   Total crashes: 16
   Period: 2021-05-10 to 2024-10-23

3. Intersection: 40.702_-73.944
   Location: 40.702, -73.944
   Total casualties: 14
   Deaths: 0
   Injuries: 14
   Total crashes: 17
   Period: 2020-02-15 to 2023-11-25

4. Intersection: 40.651_-73.956
   Location: 40.651, -73.956
   Total casualties: 14
   Deaths: 0
   Injuries: 14
   Total crashes: 18
   Period: 2020-04-23 to 2025-04-26

5. Intersection: 40.653_-74.006
   Location: 40.653, -74.006
   Total casualties: 14
   Deaths: 0
   Injuries: 14
   Total crashes: 21
   Period: 2021-01-21 to 2025-03-27

Exported to 'top_5_w

C:\Users\undop\.pyenv\pyenv-win\versions\3.12.9\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


#### Total cyclist deaths and injuries

In [30]:
# Total cyclist deaths and injuries for entire dataset
total_cyclist_deaths = gdf['NUMBER OF CYCLIST KILLED'].sum()
total_cyclist_injuries = gdf['NUMBER OF CYCLIST INJURED'].sum()
total_cyclist_casualties = gdf['cyclist_casualties'].sum()

print("Brooklyn Cyclist Casualties (2012-2025):")
print(f"Total deaths: {total_cyclist_deaths}")
print(f"Total injuries: {total_cyclist_injuries}")
print(f"Total casualties: {total_cyclist_casualties}")
print(f"Total crashes with cyclist involvement: {len(gdf[gdf['cyclist_casualties'] > 0])}")

Brooklyn Cyclist Casualties (2012-2025):
Total deaths: 71
Total injuries: 18445
Total casualties: 18516
Total crashes with cyclist involvement: 18328


#### Case study: Grand Street

Total cyclist crashes in Grand Street zone: 187

Grand Street Zone - Cyclist Crashes by Year:
Year   Crashes  Deaths   Injuries   Total   
                                    Casualties
------------------------------------------------------------
2012   8        0        8          8       
2013   17       0        17         17      
2014   21       0        21         21      
2015   14       0        14         14      
2016   14       0        14         14      
2017   14       0        14         14      
2018   15       0        15         15      
2019   11       0        11         11      
2020   15       0        15         15      
2021   15       0        15         15      
2022   11       0        11         11      
2023   15       0        16         16      
2024   6        0        7          7       
2025   11       0        11         11      

Summary for Grand Street Zone Cyclist Crashes (all years):
Total cyclist crashes: 187
Total cyclist deaths: 0
Total cyclis

In [35]:
import json
from shapely.geometry import Polygon, Point
import geopandas as gpd

# Load the Grand Street coordinates
grand_street_coords = [[-73.95141400917979,40.71133052400984],
                      [-73.94026875214871,40.71246120311044],
                      [-73.940043595441,40.711479860085866],
                      [-73.95135772000285,40.710413166480706],
                      [-73.95141400917979,40.71133052400984]]

# Create polygon from coordinates
grand_street_polygon = Polygon(grand_street_coords)

# Create a GeoDataFrame for the zone
zone_gdf = gpd.GeoDataFrame([1], geometry=[grand_street_polygon], crs='EPSG:4326')

# Convert your crash data to the same CRS if needed
gdf_wgs84 = gdf.to_crs('EPSG:4326')

# Find crashes within the Grand Street zone using spatial join
crashes_in_zone = gpd.sjoin(gdf_wgs84, zone_gdf, how='inner', predicate='within')

# Filter to only crashes with cyclist involvement
cyclist_crashes_in_zone = crashes_in_zone[crashes_in_zone['cyclist_casualties'] > 0]

print(f"Total cyclist crashes in Grand Street zone: {len(cyclist_crashes_in_zone)}")

# Group by year and count cyclist crashes
yearly_cyclist_crashes = cyclist_crashes_in_zone.groupby(cyclist_crashes_in_zone['CRASH_DATETIME'].dt.year).agg({
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum',
    'cyclist_casualties': 'sum'
}).reset_index()

# Rename columns properly
yearly_cyclist_crashes.columns = ['year', 'cyclist_deaths', 'cyclist_injuries', 'total_cyclist_casualties']

# Add crash count per year
crash_counts = cyclist_crashes_in_zone.groupby(cyclist_crashes_in_zone['CRASH_DATETIME'].dt.year).size().reset_index(name='cyclist_crash_count')
crash_counts.columns = ['year', 'cyclist_crash_count']

yearly_cyclist_crashes = yearly_cyclist_crashes.merge(crash_counts, on='year')

# Reorder columns for better CSV layout
yearly_tallies = yearly_cyclist_crashes[['year', 'cyclist_crash_count', 'cyclist_deaths', 'cyclist_injuries', 'total_cyclist_casualties']].copy()

# Export to CSV
yearly_tallies.to_csv('grand_street_zone_yearly_cyclist_tallies.csv', index=False)

print(f"\nCreated CSV with yearly tallies:")
print(yearly_tallies.to_string(index=False))
print(f"\nExported to 'grand_street_zone_yearly_cyclist_tallies.csv'")

# Also create a summary row
summary_row = {
    'year': 'TOTAL',
    'cyclist_crash_count': yearly_tallies['cyclist_crash_count'].sum(),
    'cyclist_deaths': yearly_tallies['cyclist_deaths'].sum(),
    'cyclist_injuries': yearly_tallies['cyclist_injuries'].sum(),
    'total_cyclist_casualties': yearly_tallies['total_cyclist_casualties'].sum()
}

# Create CSV with summary
yearly_tallies_with_summary = pd.concat([yearly_tallies, pd.DataFrame([summary_row])], ignore_index=True)
yearly_tallies_with_summary.to_csv('grand_street_zone_yearly_cyclist_tallies_with_summary.csv', index=False)

print(f"\nAlso created 'grand_street_zone_yearly_cyclist_tallies_with_summary.csv' with totals row")

Total cyclist crashes in Grand Street zone: 187

Created CSV with yearly tallies:
 year  cyclist_crash_count  cyclist_deaths  cyclist_injuries  total_cyclist_casualties
 2012                    8               0                 8                         8
 2013                   17               0                17                        17
 2014                   21               0                21                        21
 2015                   14               0                14                        14
 2016                   14               0                14                        14
 2017                   14               0                14                        14
 2018                   15               0                15                        15
 2019                   11               0                11                        11
 2020                   15               0                15                        15
 2021                   15               0      